# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [173]:
%%capture

#Instalando o tweepy
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [174]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [175]:
#Dados de autenticação do twitter:

# @manoelaclc

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [187]:
#Produto escolhido:
produto = 'Correios'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [183]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [184]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [188]:
import mpmath

dadosTre = pd.read_excel('correios.xlsx',sheetname="Treinamento")
dadosTes = pd.read_excel('correios.xlsx',sheetname="Teste")

In [189]:
import numpy as np
todatreN = np.sum(dadosTre[dadosTre.classificador=="neutro"].Treinamento + " ")
todatreR = np.sum(dadosTre[dadosTre.classificador=="relevante"].Treinamento + " ")
todatre = np.sum(dadosTre.Treinamento + " ")

import string
#def para retirar pontuação
exclude = set(string.punctuation)
def SemP(toda):
    toda = toda.split()
    lista = []
    for s in toda:
        for i in s:
            if i in exclude:
                s = s.replace(i,'') 
        lista.append(s)
    return lista   
    
TTreN = SemP(todatreN)
TTreR = SemP(todatreR)
TTre = SemP(todatre)
TTreR

['esperando',
 'uma',
 'empresa',
 'de',
 'entrega',
 'surgir',
 'e',
 'competir',
 'com',
 'o',
 'correios',
 'a',
 'ponto',
 'deles',
 'melhorarem',
 'a',
 'entrega',
 'e',
 'largarem',
 'de',
 'sacanear',
 'com',
 'o',
 'consumidor',
 'compra',
 'aprovada',
 'na',
 'sexta',
 'é',
 'uma',
 'lástima',
 'ne',
 'o',
 'produto',
 'só',
 'é',
 'postado',
 'na',
 'segunda',
 'e',
 'o',
 'correios',
 'só',
 'libera',
 'na',
 'quarta',
 'fico',
 'até',
 'triste',
 'rt',
 'heymika',
 'você',
 'ganha',
 'um',
 'produto',
 'que',
 'custa',
 '5000',
 'ienes',
 'menos',
 'de',
 '150',
 'reais',
 'os',
 'correios',
 'taxam',
 'em',
 '140',
 'reais',
 'obg',
 'correios',
 'por',
 'nunca',
 'entregar',
 'nada',
 'no',
 'prazo',
 'o',
 'correios',
 'de',
 'itapecerica',
 'é',
 'tão',
 'ruim',
 'no',
 'atendimento',
 'que',
 'os',
 'caras',
 'desligaram',
 'o',
 'telefone',
 'por',
 'conta',
 'das',
 'reclamações',
 'acordo',
 'às',
 '9h',
 'p',
 'estar',
 'pronto',
 'p',
 'receber',
 'encomenda',
 'e

In [190]:
TreN = pd.Series(TTreN) # Todas as palavras em neutro
TreR = pd.Series(TTreR) # Todas as palavras em relevante
Tre = pd.Series(TTre)

In [191]:
PP_N = TreN.value_counts()
PP_R = TreR.value_counts()
P_dif = len(Tre.value_counts())
PP_R

correios              92
que                   39
de                    38
e                     36
a                     35
o                     33
é                     27
entrega               23
um                    18
dos                   18
rt                    16
pra                   16
os                    15
em                    14
uma                   14
eu                    12
meu                   12
encomenda             12
q                     12
não                   12
minha                 11
mais                  10
com                   10
no                    10
do                     9
empresa                9
na                     9
por                    9
tá                     9
mas                    8
                      ..
gritando               1
httpstcobs9naro6yo     1
traz                   1
aparecer               1
taxi                   1
ranner                 1
status                 1
kkkkkk                 1
pronto                 1


In [192]:
Tot = pd.DataFrame(dadosTre.classificador.value_counts(True)) #prob total neutro e relevante
P_N = Tot.classificador[0] #prob total neutro
P_R = Tot.classificador[1] #prob total rele
T_N = len(dadosTre[dadosTre.classificador=="neutro"]) #Quantidade de neutros
T_R = len(dadosTre[dadosTre.classificador=="relevante"]) #Quantidade de relevantes
Tot

,classificador
neutro,0.693333
relevante,0.306667


In [193]:
P_P_R = (PP_R+1)/(T_R + P_dif) #lista com probabilidade das palavras quando tw relevante
P_P_N = (PP_N+1)/(T_N + P_dif) #lista com probabilidade das palavras quando tw neutro
P_P_R

correios              0.060468
que                   0.026008
de                    0.025358
e                     0.024057
a                     0.023407
o                     0.022107
é                     0.018205
entrega               0.015605
um                    0.012354
dos                   0.012354
rt                    0.011053
pra                   0.011053
os                    0.010403
em                    0.009753
uma                   0.009753
eu                    0.008453
meu                   0.008453
encomenda             0.008453
q                     0.008453
não                   0.008453
minha                 0.007802
mais                  0.007152
com                   0.007152
no                    0.007152
do                    0.006502
empresa               0.006502
na                    0.006502
por                   0.006502
tá                    0.006502
mas                   0.005852
                        ...   
gritando              0.001300
httpstco

In [194]:
teste = dadosTes.Teste

#retira pontuação do teste
exclude = set(string.punctuation)
def SemPT(teste):
    testeL = []
    for n in range(len(teste)):
        frase = teste[n].split()
        fraseL = []
        for s in frase:
            for i in s:
                if i in exclude:
                    s = s.replace(i,'') 
            fraseL.append(s)  
        testeL.append(fraseL)
    return testeL

#calcula probabilidade da frase - neutro ou relevante
def ProbF(listafrase,P_P,P_,T_):
    listaP = []
    for frase in listafrase:
        P_frase = mpmath.mpf(1.000)
        for p in frase:
            if p in P_P:
                P_frase = P_frase * P_P[p] * P_
            else:
                P_frase = mpmath.mpf(1)/(T_+P_dif) * P_frase * P_
        listaP.append(P_frase)
    return listaP

#cria lista sem pontuação
testeL = SemPT(teste)
#Cálculo da probabilidade do tweet ser neutro
teste_P_N = ProbF(testeL,P_P_N,P_N,T_N)
#Cálculo da probabilidade do tweet ser relevante
teste_P_R = ProbF(testeL,P_P_R,P_R,T_R)

#Compara as listas de probabilidade e devolve neutro ou relevante
def Comparador(teste_P_N,teste_P_R):
    resp = []
    for i in range(len(teste_P_N)):
        if teste_P_N[i] > teste_P_R[i]:
            resp.append('neutro')
        elif teste_P_N[i] < teste_P_R[i]:
            resp.append('relevante')
    return resp

#lista de respostas do aplicativo
resp_teste = Comparador(teste_P_N,teste_P_R)

len(resp_teste)

200

In [195]:
data = {'twitters': testeL, 'classificador': dadosTes.classificador, 'respostas': resp_teste}

In [196]:
tabelaT = pd.DataFrame(data, columns = ['twitters','classificador','respostas'])
tabelaT

,twitters,classificador,respostas
0,"[pf, prende, homem, acusado, de, roubo, a, agê...",neutro,neutro
1,"[im, at, correios, httpstcotqaeppqdx5]",neutro,neutro
2,"[rt, emersontsilva, se, precisar, de, converso...",neutro,neutro
3,"[depois, de, 1, mês, e, 15, dias, a, pessoa, v...",relevante,neutro
4,"[cjcastro45, está, cheio, de, funcionários, de...",neutro,neutro
5,"[ihaaaaa, correios, chegouuuu, lt333]",neutro,neutro
6,"[pabllovittar, vem, responsabilidade, em, casa...",neutro,neutro
7,"[bilhetes, para, a, taça, dos, correios, a, pa...",neutro,neutro
8,"[devolve, minha, manhã, e, tarde, de, sexta, c...",relevante,neutro
9,"[porque, o, correios, demora, tanto, pra, traz...",relevante,neutro


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [197]:
verificacao = []
for i in range(len(resp_teste)):
    if dadosTes.classificador[i] == resp_teste[i]:
        if resp_teste[i] == 'neutro':
            verificacao.append('negativo verdadeiro')
        elif resp_teste[i] == 'relevante':
            verificacao.append('positivo verdadeiro')
            
    elif dadosTes.classificador[i] != resp_teste[i]:
        if resp_teste[i] == 'neutro':
            verificacao.append('negativo falso')
        elif resp_teste[i] == 'relevante':
            verificacao.append('positivo falso')
    

In [198]:
verificacaoS = pd.Series(verificacao)
porcentagem = verificacaoS.value_counts(True)
porcentagem

negativo verdadeiro    0.690
negativo falso         0.275
positivo verdadeiro    0.035
dtype: float64

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


   Os resultados do teste com a planilha dos correios mostram que a probabilidade de um twitter ser 
neutro é muito grande, o que influencia muito nos resultados do classificador. A porcentagem
de positivos falsos é 0 por esse motivo. Esse resultado nulo também tem relação com sarcasmos e duplos negativos, que
não são corretamente interpretados pelo sistema,pelo fato de a formação da frase não ser considerada na conta.

   É possível expandir o projeto e pensar em reclamações mais específicas, como pro exemplo os atrasos. Assim, é
possível organizar melhor a base de dados e obter resultados mais precisos.